<a href="https://colab.research.google.com/github/YounSooKimTech/NLP_teamproject/blob/main/IMDB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis



In [2]:
# get first page data
api_key = "50b3d92df451f49698a9387ad0edf464" # get TMDB API key 
response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&language=en-US'+'&page='+str(1))


# sampling for code

In [8]:
films = response.json() 
films = films['results']
# create data frame for first page
films = pd.DataFrame(films)

films.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/gHbSWwczk3pipdqfTRnhOXpqQfL.jpg,[27],19158,en,Graveyard Shift,John Hall is a drifter who wanders into a smal...,13.468,/oVoY9FlZ3soX6g1Fur5jBOZsVVw.jpg,1990-10-26,Graveyard Shift,False,5.2,189
1,False,/l9nnZoRfJdAcU5pBKNGj5ZvJZdr.jpg,"[10402, 99]",806439,es,Morat: Balas Perdidas,"Morat, the band with the highest amount of tic...",13.467,/1Thyu0UorpClFHHpLt2d0h6k6Wf.jpg,2021-03-26,Morat: Balas Perdidas,False,9.2,43
2,False,/nkUbnGJZI5Pv9V58hUTgjrwc7tO.jpg,"[37, 12, 28, 18, 10749]",995,en,Stagecoach,A group of people traveling on a stagecoach fi...,13.467,/kXJISFExDX9MbI9YzJznvUs9unY.jpg,1939-03-03,Stagecoach,False,7.7,882
3,False,/jA0KwNRRQWbQzxo4JQq4YkpoePp.jpg,"[28, 18, 53]",11123,en,Earthquake,Various interconnected people struggle to surv...,13.466,/6hJorsInxNpj1dIekh6dK6Kh9S6.jpg,1974-01-01,Earthquake,False,6.1,256
4,False,/rH5st8lYd2Jfhk3ixN4mluYDFH7.jpg,"[28, 35]",6916,it,...altrimenti ci arrabbiamo!,"After a tied 1st place in a local stunt race, ...",13.466,/nsnVnZDqlpe6CHENnapq19RGRzC.jpg,1974-03-29,"Watch Out, We're Mad",False,7.4,671


In [14]:
df = films[["title", "overview"]]
df["text"] = df["title"] + df["overview"]
df["text"] = df["text"].str.lower()
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,title,overview,text
0,Graveyard Shift,John Hall is a drifter who wanders into a smal...,graveyard shiftjohn hall is a drifter who wand...
1,Morat: Balas Perdidas,"Morat, the band with the highest amount of tic...","morat: balas perdidasmorat, the band with the ..."
2,Stagecoach,A group of people traveling on a stagecoach fi...,stagecoacha group of people traveling on a sta...
3,Earthquake,Various interconnected people struggle to surv...,earthquakevarious interconnected people strugg...
4,"Watch Out, We're Mad","After a tied 1st place in a local stunt race, ...","watch out, we're madafter a tied 1st place in ..."


In [24]:
# tokenization
import spacy

nlp = spacy.load("en_core_web_sm")

df["tokens"] = df["text"].apply(lambda x: [t.text for t in nlp.tokenizer(x)])
df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,overview,text,tokens
0,Graveyard Shift,John Hall is a drifter who wanders into a smal...,graveyard shiftjohn hall is a drifter who wand...,"[graveyard, shiftjohn, hall, is, a, drifter, w..."
1,Morat: Balas Perdidas,"Morat, the band with the highest amount of tic...","morat: balas perdidasmorat, the band with the ...","[morat, :, balas, perdidasmorat, ,, the, band,..."


# whole dataset

In [3]:
films = response.json() 
films = films['results']
# create data frame for first page
films = pd.DataFrame(films)

In [4]:
# Retriving data by looping through 500 pages 
for x in range (2,500,1):
  response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key +'&page='+str(x))
  re = response.json()
  results = re['results']
  df = pd.DataFrame(results)
  films = films.append(df)


In [5]:
len(films)

9980

In [6]:
films.head(50)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/yYrvN5WFeGYjJnRzhY0QXuo4Isw.jpg,"[28, 12, 878]",505642,en,Black Panther: Wakanda Forever,"Queen Ramonda, Shuri, M’Baku, Okoye and the Do...",4594.775,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg,2022-11-09,Black Panther: Wakanda Forever,False,7.6,465
1,False,/y5Z0WesTjvn59jP6yo459eUsbli.jpg,"[27, 53]",663712,en,Terrifier 2,"After being resurrected by a sinister entity, ...",4528.571,/b6IRp6Pl2Fsq37r9jFhGoLtaqHm.jpg,2022-10-06,Terrifier 2,False,7.0,579
2,False,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg,"[28, 14, 878]",436270,en,Black Adam,"Nearly 5,000 years after he was bestowed with ...",4328.431,/3zXceNTtyj5FLjwQXuPvLYK5YYL.jpg,2022-10-19,Black Adam,False,6.9,1058
3,False,/u9Io1hVnxb2b3T0z99aOK8sq0SN.jpg,"[9648, 12, 80]",829280,en,Enola Holmes 2,Now a detective-for-hire like her infamous bro...,4081.058,/tegBpjM5ODoYoM1NjaiHVLEA0QM.jpg,2022-11-04,Enola Holmes 2,False,7.7,761
4,False,/b2FxWOxe9K7ZZ1uaPOze2RJ1ajq.jpg,"[27, 28, 35]",675054,es,MexZombies,A group of teenagers must face a zombie apocal...,3604.019,/85zufUxhD97k2s5Bu2u101Qd8Sg.jpg,2022-10-26,MexZombies,False,7.3,66
5,False,/mqsPyyeDCBAghXyjbw4TfEYwljw.jpg,"[10752, 18, 28]",49046,de,Im Westen nichts Neues,"Paul Baumer and his friends Albert and Muller,...",3142.726,/hYqOjJ7Gh1fbqXrxlIao1g8ZehF.jpg,2022-10-07,All Quiet on the Western Front,False,7.8,726
6,False,/pGx6O6IwqADOsgmqWzPysmWnOyr.jpg,"[28, 14]",732459,en,Blade of the 47 Ronin,"In this sequel to ""47 Ronin,"" a new class of w...",2516.902,/kjFDIlUCJkcpFxYKtE6OsGcAfQQ.jpg,2022-10-25,Blade of the 47 Ronin,False,6.8,66
7,False,/1DBDwevWS8OhiT3wqqlW7KGPd6m.jpg,[53],985939,en,Fall,"For best friends Becky and Hunter, life is all...",2180.037,/spCAxD99U1A6jsiePFoqdEcY0dG.jpg,2022-08-11,Fall,False,7.3,1715
8,False,/5GA3vV1aWWHTSDO5eno8V5zDo8r.jpg,"[27, 53]",760161,en,Orphan: First Kill,After escaping from an Estonian psychiatric fa...,1924.954,/pHkKbIRoCe7zIFvqan9LFSaQAde.jpg,2022-07-27,Orphan: First Kill,False,6.8,1281
9,False,/tIX6j3NzadlwGcJ52nuWdmtOQkg.jpg,"[27, 53, 9648]",717728,en,Jeepers Creepers: Reborn,Forced to travel with her boyfriend to a horro...,1909.338,/aGBuiirBIQ7o64FmJxO53eYDuro.jpg,2022-09-15,Jeepers Creepers: Reborn,False,5.8,452


# Preprocessing

In [7]:
films[["id","genre_ids", "title", "overview","adult"]]

KeyboardInterrupt: ignored

In [ ]:
# make a string

films["text"] = films["title"] + films["overview"]
films.head()

In [ ]:
df = films[["id","genre_ids", "title", "overview","adult"]]
df.head(50)